In [ ]:
import nltk, string, gensim
from nltk.corpus import abc
nltk.download('abc')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package abc to /root/nltk_data...
[nltk_data]   Unzipping corpora/abc.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
abc.sents()

[['PM', 'denies', 'knowledge', 'of', 'AWB', 'kickbacks', 'The', 'Prime', 'Minister', 'has', 'denied', 'he', 'knew', 'AWB', 'was', 'paying', 'kickbacks', 'to', 'Iraq', 'despite', 'writing', 'to', 'the', 'wheat', 'exporter', 'asking', 'to', 'be', 'kept', 'fully', 'informed', 'on', 'Iraq', 'wheat', 'sales', '.'], ['Letters', 'from', 'John', 'Howard', 'and', 'Deputy', 'Prime', 'Minister', 'Mark', 'Vaile', 'to', 'AWB', 'have', 'been', 'released', 'by', 'the', 'Cole', 'inquiry', 'into', 'the', 'oil', 'for', 'food', 'program', '.'], ...]

In [ ]:
model= gensim.models.Word2Vec(abc.sents())

In [ ]:
data=model.wv.most_similar('science')
print(data)

[('law', 0.9415050745010376), ('agriculture', 0.931178867816925), ('general', 0.927341103553772), ('policy', 0.9263863563537598), ('media', 0.9183502197265625), ('practice', 0.9176878929138184), ('discussion', 0.9153490662574768), ('reservoir', 0.9141187071800232), ('Crean', 0.9128280878067017), ('tight', 0.9121520519256592)]


In [ ]:
import json
json_file ='intents.json'
with open('intents.json','r') as f:
    data = json.load(f)

In [ ]:
data

[{'tag': 'welcome',
  'patterns': ['Hi',
   'How are you',
   'Is any one to talk?',
   'Hello',
   'hi are you available'],
  'responses': ['Hello, thanks for contacting us',
   'Good to see you here',
   ' Hi there, how may I assist you?']},
 {'tag': 'goodbye',
  'patterns': ['Bye', 'See you later', 'Goodbye', 'I will come back soon'],
  'responses': ['See you later, thanks for visiting',
   'have a great day ahead',
   'Wish you Come back again soon.']},
 {'tag': 'thankful',
  'patterns': ['Thanks for helping me',
   'Thank your guidance',
   "That's helpful and kind from you"],
  'responses': ['Happy to help!',
   'Any time!',
   'My pleasure',
   'It is my duty to help you']},
 {'tag': 'hoursopening',
  'patterns': ['What hours are you open?',
   'Tell your opening time?',
   'When are you open?',
   'Just your timing please'],
  'responses': ["We're open every day 8am-7pm",
   'Our office hours are 8am-7pm every day',
   'We open office at 8 am and close at 7 pm']},
 {'tag': 'pay

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df['patterns'] = df['patterns'].apply(', '.join) 

In [ ]:
from nltk.corpus import stopwords
from textblob import Word
stop = stopwords.words('english')
df['patterns'] = df['patterns'].apply(
    lambda x:' '.join(x.lower() for x in x.split()))
df['patterns']= df['patterns'].apply(
    lambda x: ' '.join(x for x in x.split() if x not in string.punctuation))
df['patterns']= df['patterns'].str.replace('[^\w\s]','')
df['patterns']= df['patterns'].apply(
    lambda x: ' '.join(x for x in x.split() if  not x.isdigit()))
df['patterns'] = df['patterns'].apply(
    lambda x:' '.join(x for x in x.split() if not x in stop))
df['patterns'] = df['patterns'].apply(
    lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

<ipython-input-12-a10f260ff0b5>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['patterns']= df['patterns'].str.replace('[^\w\s]','')


In [ ]:
from gensim.models import Word2Vec

In [ ]:
#taking the outer list
bigger_list=[]
for i in df['patterns']:
    li = i.split()
    bigger_list.append(li)

In [ ]:
#custom data is fed to machine for further processing
model = Word2Vec(bigger_list, min_count=1,size=300,workers=4)
print(model)

Word2Vec(vocab=32, size=300, alpha=0.025)


In [ ]:
model.save("word2vec.model")
model.save("model.bin")

In [ ]:
new_model = Word2Vec.load('model.bin')

In [ ]:
vocab = list(new_model.wv.vocab)     # new_model.wv.key_to_index
vocab;

In [ ]:
similar_words = new_model.wv.most_similar('thanks')	
print(similar_words)	

[('pay', 0.09113314002752304), ('hour', 0.09038735926151276), ('thats', 0.08777609467506409), ('back', 0.07375020533800125), ('mastercard', 0.06709612905979156), ('using', 0.04830966144800186), ('guidance', 0.047691710293293), ('please', 0.04701085388660431), ('soon', 0.042033106088638306), ('thank', 0.04159191623330116)]


In [ ]:
similar_words = new_model.wv.most_similar('available')	
print(similar_words[:3])	

[('please', 0.09661215543746948), ('mastercard', 0.09072067588567734), ('hello', 0.08574282377958298)]


In [ ]:
similar_words = new_model.wv.most_similar('pay')	
print(similar_words[:3])

[('come', 0.12683022022247314), ('cash', 0.11225173622369766), ('thank', 0.09228657186031342)]


In [ ]:
dissimlar_words = model.wv.doesnt_match('see you later, thanks for visiting'.split())
print(dissimlar_words)

see


In [ ]:
!pip install ktrain flask